# Predicción de unidades vendidas productos RDS para el resto del 2025

## Modelo ARIMA

In [1]:
# Librerías
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
# Base de datos
df_modelo = pd.read_excel("/Users/martincarrasco/Desktop/Martín_Carrasco/Reportes/Interanuales/Cuentas RDS/Ventas Consolidado RDS 2024-2025 (hasta 15-06-2025).xlsx", sheet_name = "Hoja1")
df_modelo

,Titulo_Publicacion,Marca,Categoría,Subcategoría,Año,Mes,Fecha_Compra,Nombre Vendedor,Unidades_Vendidas,Volumen_de_Ventas_Moneda_Local
0,Culata De Motor Ford Ranger 3.2 2013 201,FABRICAS CHINAS,Motores,Componentes de Motor,2024,10,2024-10-31,REICARSCHILE,9,5394519.0
1,Cremallera Dirección Hidráulica Chevrole,YOKOMITSU,Suspensión y Dirección,Dirección Hidráulica,2024,1,2024-01-31,REPUESTOS DEL SOL,41,4919590.0
2,Kit Embrague Con Volante Ssangyong Actyo,VALEO,Transmisión,Embragues,2024,11,2024-11-30,BLACKPARTSCL,12,4656840.0
3,Kit Embrague Ssangyong Actyon Sport 2.0,VALEO,Transmisión,Embragues,2024,6,2024-06-30,BLACKPARTSCL,28,4585462.0
4,Kit Embrague Automatizado Ford Focus 2.0,LUK,Transmisión,Embragues,2024,11,2024-11-30,BLACKPARTSCL,4,3933562.0
...,...,...,...,...,...,...,...,...,...,...
160184,Filtro Aire Para Citroen C3 1.4 2003 200,STP,Filtros,Filtros de Aire,2025,1,2025-01-31,INDUSOLRDS,1,4990.0
160185,Plumilla Del Izq Para Toyota Urban Cruis,STP,Repuestos de Exterior,Plumillas Limpiaparabrisas,2025,3,2025-03-31,INDUSOLRDS,1,4990.0
160186,Filtro Aire Para Volkswagen Golf A3 1.8,WURTEX,Filtros,Filtros de Aire,2025,4,2025-04-30,INDUSOLRDS,1,4990.0
160187,Manilla Interior Gris Para Chevrolet Luv,DANAHER,Carrocería,Manillas,2025,6,2025-06-30,MERCADOREPUESTOSCL,1,4741.0


In [86]:
marcas = df_modelo["Marca"].unique()
for m in sorted(marcas):
    print(m)

03303839 90373164
06A919501
091230327
3RG
4X4
7L2Z-1104-A
9956288
A&I
A&M
A-GIST
AB FRICTION
ABA
ACAR - CHINA
ACCENT
ACHR
ADIOU
AIRTEX
AKRON
AL- 010760256
ALEMANIA
ALKAR
ALS
ALT
ALTERNATIVA
ALTERNATIVA COREANA
ALTERNATIVA.
ALTERNATIVAS
ALTERNATIVO
ALTERNATIVOS
ALTRNATIVO
AMC
ANJUN
ANKHEI
AP!
APA- MEXICO
APEX
API
APK
APK CHINA
ARCO
ARCO ( TAIWAN )
ARROW
ASHIMORI
ATSUKI
AUDI
AUTOLEK
AUTOPLUS
AUTOTEC
AUTOTOP
AWZ
AYFAR
BBB
BEN
BERU - ALEMANIA
BESER
BESTE
BGN
BHORKE
BLAWER
BLUEPRINT
BOGR
BOSCH
BOSSUN
BRASIL
BRAZIL
BREMBO
BROALAU
BVP GENUINE
BVP-GENUINE
C&H
C.T.I
C1-M
CADIC
CAHSA
CAR-DEX
CARTER
CAS
CAS - CHINA
CASP
CASP - CHINA
CAUPLAS
CAZ U.E.  - ROD: ALEMAN
CB
CENTURY
CESAR AMOROSO
CHANGAN
CHERRY
CHEVROLET
CHINA
CHINAS
CHINAY
CHINO
CHM
CHUAN AO
CI-M
CIB
CIM
CITROËN
CLASSIC
COFAP
COGEFA
COLOR NEGRO
CONTIL
CONTINENTAL
CORAM
CR-YS201037
CTR
CTR KOREA
D-MARK
D.S.
DAEWHA
DAEWOO
DAIDO
DAIHATSU
DANAHER
DANAHER - CHINA
DANYANG HONGFENG
DASHI
DCROER
DEAJIN
DELUX
DENNO
DEPO
DFDFSK
DIAMOND
DIFORZA
DI

In [87]:
df_modelo.rename(columns={"Titulo_Publicacion": "Producto"}, inplace = True)

In [88]:
df_modelo = df_modelo[["Producto", "Marca", "Fecha_Compra", "Unidades_Vendidas"]]
df_modelo

,Producto,Marca,Fecha_Compra,Unidades_Vendidas
1,Cremallera Dirección Hidráulica Chevrole,YOKOMITSU,2024-01-31,41
5,Cremallera Dirección Hidráulica Chevrole,GENÉRICO,2024-12-31,37
9,Kit Distribucion Mitsubishi L200 2.5 200,"MITSUBOSHI, GMB, VALEO",2024-10-31,32
13,Cremallera Dirección Hidráulica Chevrole,YOKOMITSU,2024-03-31,24
14,Kit Distribucion Mitsubishi L200 2.5 200,"MITSUBOSHI, GMB",2024-06-30,30
...,...,...,...,...
160164,Filtro Aceite Para Subaru Impreza 1.6 Ej,WURTEX,2025-03-31,1
160167,Filtro Polen Chevrolet Spark Gt 1.2 B12d,ALTERNATIVA,2025-04-30,1
160174,Empaquetadura Tapa Valvula Para Changan,SANWA,2025-04-30,1
160176,Filtro Aceite Para Nissan X-trail 2.5 Qr,MANDO,2025-04-30,1


In [89]:
df_modelo = df_modelo.groupby([
    "Producto",
    "Marca",
    pd.Grouper(key="Fecha_Compra", freq="M")
], as_index=False)["Unidades_Vendidas"].sum()

/var/folders/lr/rkgxn4r50h712qndyykwjmfh0000gp/T/ipykernel_23174/320908091.py:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  pd.Grouper(key="Fecha_Compra", freq="M")


In [90]:
df_modelo["Mes"] = df_modelo["Fecha_Compra"].dt.to_period("M")
total_meses = df_modelo["Mes"].nunique()
meses_por_producto = df_modelo.groupby("Producto")["Mes"].nunique()
productos_completos = meses_por_producto[meses_por_producto == total_meses].index
df_modelo = df_modelo[df_modelo["Producto"].isin(productos_completos)]

df_modelo

,Producto,Marca,Fecha_Compra,Unidades_Vendidas,Mes
3952,Bandeja Inferior Para Ssangyong Actyon 4,GENÉRICO,2024-01-31,14,2024-01
3953,Bandeja Inferior Para Ssangyong Actyon 4,GENÉRICO,2024-02-29,8,2024-02
3954,Bandeja Inferior Para Ssangyong Actyon 4,GENÉRICO,2024-03-31,6,2024-03
3955,Bandeja Inferior Para Ssangyong Actyon 4,GENÉRICO,2024-04-30,10,2024-04
3956,Bandeja Inferior Para Ssangyong Actyon 4,GENÉRICO,2024-05-31,2,2024-05
...,...,...,...,...,...
40666,Resorte Espiral Delantero Para Toyota Ya,WURTEX,2025-02-28,3,2025-02
40667,Resorte Espiral Delantero Para Toyota Ya,WURTEX,2025-03-31,5,2025-03
40668,Resorte Espiral Delantero Para Toyota Ya,WURTEX,2025-04-30,1,2025-04
40669,Resorte Espiral Delantero Para Toyota Ya,WURTEX,2025-05-31,4,2025-05


In [91]:
producto = "Resorte Espiral Delantero Para Toyota Ya"
marca = "WURTEX"

df_filtrado = df_modelo[
    (df_modelo["Producto"] == producto) &
    (df_modelo["Marca"] == marca)
].sort_values("Fecha_Compra")

In [92]:
train = df_filtrado[df_filtrado["Fecha_Compra"].dt.year == 2024].set_index("Fecha_Compra")["Unidades_Vendidas"]

test = df_filtrado[
    (df_filtrado["Fecha_Compra"] >= "2025-01-31") & 
    (df_filtrado["Fecha_Compra"] <= "2025-05-31")
].set_index("Fecha_Compra")["Unidades_Vendidas"]

In [93]:
train

Fecha_Compra
2024-01-31    4
2024-02-29    3
2024-03-31    4
2024-04-30    3
2024-05-31    2
2024-06-30    2
2024-07-31    4
2024-08-31    4
2024-09-30    2
2024-10-31    3
2024-11-30    6
2024-12-31    1
Name: Unidades_Vendidas, dtype: int64

In [94]:
test

Fecha_Compra
2025-01-31    1
2025-02-28    3
2025-03-31    5
2025-04-30    1
2025-05-31    4
Name: Unidades_Vendidas, dtype: int64

In [95]:
model = ARIMA(train, order=(1, 1, 1))
model_fit = model.fit()

/Users/martincarrasco/Desktop/Martín_Carrasco/rds-analisis/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ME will be used.
  self._init_dates(dates, freq)
/Users/martincarrasco/Desktop/Martín_Carrasco/rds-analisis/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ME will be used.
  self._init_dates(dates, freq)
/Users/martincarrasco/Desktop/Martín_Carrasco/rds-analisis/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ME will be used.
  self._init_dates(dates, freq)


In [96]:
forecast = model_fit.forecast(steps=5)
forecast.index = pd.date_range(start="2025-01-31", periods=5, freq="M")

/var/folders/lr/rkgxn4r50h712qndyykwjmfh0000gp/T/ipykernel_23174/2591955870.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  forecast.index = pd.date_range(start="2025-01-31", periods=5, freq="M")


In [97]:
resultado = pd.DataFrame({
    "Real": test.values,
    "Predicción": forecast
}, index=forecast.index)

mae = mean_absolute_error(resultado["Real"], resultado["Predicción"])
rmse = mean_squared_error(resultado["Real"], resultado["Predicción"])
mape = (abs(resultado["Real"] - resultado["Predicción"]) / resultado["Real"]).mean() * 100

print("Evaluación del modelo ARIMA:")
print(f"MAE  : {mae:,.2f}")
print(f"RMSE : {rmse:,.2f}")
print(f"MAPE : {mape:.2f}%")
print("\n🔍 Resultado:")
print(resultado)

Evaluación del modelo ARIMA:
MAE  : 1.54
RMSE : 3.40
MAPE : 113.62%

🔍 Resultado:
            Real  Predicción
2025-01-31     1    3.952949
2025-02-28     3    2.935813
2025-03-31     5    3.286163
2025-04-30     1    3.165486
2025-05-31     4    3.207053
